In [1]:
from subprocess import call
import glob
import os
import pandas as pd
import numpy as np
import sqlite3 as sq
from sqlalchemy import create_engine
pd.options.display.max_columns = 999

In [ ]:
years = range(2001,2017)

In [9]:
#grants
for y in years:
    url = 'https://exporter.nih.gov/CSVs/final/RePORTER_PRJ_C_FY' + str(y) + '.zip'
    call(['wget', url])
#2017
for y in range(1,10):
    url = 'https://exporter.nih.gov/CSVs/final/RePORTER_PRJ_C_FY2017_00' + str(y) + '.zip'
    call(['wget', url])
for y in range(0,9):
    url = 'https://exporter.nih.gov/CSVs/final/RePORTER_PRJ_C_FY2017_01' + str(y) + '.zip'
    call(['wget', url])
#publications
for y in years:
    url = 'https://exporter.nih.gov/CSVs/final/RePORTER_PUB_C_' + str(y) + '.zip'
    call(['wget', url])
#link_table
for y in years:
    url = 'https://exporter.nih.gov/CSVs/final/RePORTER_PUBLNK_C_' + str(y) + '.zip'
    call(['wget', url])

In [12]:
!unzip \*.zip

In [25]:
prj_files  = glob.glob('RePORTER_PRJ*.*')
pub_files  = glob.glob('RePORTER_PUB_C*.*')
link_files = glob.glob('RePORTER_PUBLNK_C_*.*')

In [2]:
con = sq.connect('nih.db')

In [ ]:
def appendTables(fns, tableName):
    lst_file = fns.pop()
    df = pd.read_csv(lst_file, header=0, encoding="iso-8859-1")
    all_cols = set(df.columns);
    df.to_sql(tableName, con, flavor='sqlite', if_exists='replace', index=False)
    for f in fns:
        df_a = pd.read_csv(f, header = 0, encoding="iso-8859-1")
        col = set(df_a.columns);
        miss_cols = all_cols - col;
        print(f)
        if len(miss_cols) !=0:
            data = np.empty((len(df), len(miss_cols)),dtype=object)
            df_new = pd.DataFrame(data, columns=miss_cols)
            df_a = pd.concat([df_a, df_new], axis=1)
        df_a.to_sql(tableName, con, flavor='sqlite', if_exists='append', index=False)
appendTables(prj_files, 'projects')
appendTables(pub_files, 'publication')
appendTables(link_files, 'link')




In [ ]:
con.execute('create index link_PMID_index on link(PMID);')
con.execute('create index link_PROJECT_NUMBER_index on link(PROJECT_NUMBER);')
con.execute('create index link_PMID_index on publication(PMID);')
con.execute('create index publication_PMID_index on publication(PMID);')
